# Overview of the use case

---------(Overview on the use case,
why is it important for data producers,
usefulness of the library
when reading non-SDMX data
and convert to SDMX. Mention validations and calculations over VTL.)---------

Talking points and agenda:

- General use of pysdmx on Data Producers
- Outside the SDMX garden, looking at LEI and GLEIF
- Data cleaning and set up using pandas
- Downloading and reading the ConceptScheme on SDMX-ML 2.1 using read_sdmx
- Retrieving the Schema from FMR (FusionJSON)
- Convert the Schema to a VTL DataStructure
- Using VTL to validate the data
- Using VTL to perform calculations
- Generate SDMX-ML file with the aggregated data
- Reading back the SDMX file using read_sdmx

### List of pysdmx classes and functions used in this notebook:

Functions:
- pysdmx.io.read_sdmx
- pysdmx.io.csv.sdmx20.writer.write

Classes:
- pysdmx.api.fmr.RegistryClient (and methods)
- pysdmx.model.message.Message (and methods)
- pysdmx.io.pd.PandasDataset
- pysdmx.model.dataflow.Schema

# Outside the SDMX garden, looking at LEI and GLEIF

---------(Explanation on LEI and GLEIF, use for data producers)---------

## Data cleaning and set up using pandas and pysdmx

For this use case,
we will use the Golden Copy file from GLEIF (link)
and filter on those LEI with status Active. 

The code renames the columns and select the data we need for later validation.
We added the possibility of saving the data into plain CSV or SDMX-CSV 2.0
(using pysdmx).

The code uses the chunking capabilities of Pandas for better memory efficiency.
This is a prototype of the streaming capabilities with pandas in pysdmx,
which will be available by the end of 2025.

For the sake of this example,
we distinguish between the Golden Copy original path
(link to download the file) and the Golden Copy Changed,
which would be the output of this code.

This code requires to install the extra data from pysdmx,
which simply install pandas.

```bash
pip install pysdmx[data]
```

In [36]:
import os

import pandas as pd
from pysdmx.io.csv.sdmx20.writer import write
from pysdmx.io.pd import PandasDataset

# Original columns and their simple name for next steps of this tutorial
RENAME_DICT = {
    "LEI": "LEI",
    "Entity.LegalName": "LEGAL_NAME",
    "Entity.LegalAddress.Country": "COUNTRY_INCORPORATION",
    "Entity.HeadquartersAddress.Country": "COUNTRY_HEADQUARTERS",
    "Entity.EntityCategory": "CATEGORY",
    "Entity.EntitySubCategory": "SUBCATEGORY",
    "Entity.LegalForm.EntityLegalFormCode": "LEGAL_FORM",
    "Entity.EntityStatus": "STATUS",
    "Entity.LegalAddress.PostalCode": "POSTAL_CODE",
}


def _process_chunk(data: pd.DataFrame):
    data.rename(columns=RENAME_DICT, inplace=True)
    data = data[list(RENAME_DICT.values())]
    data = data[data["STATUS"] == "ACTIVE"]
    del data["STATUS"]
    return data


def _save_as_sdmx_csv(data: pd.DataFrame):
    dataset = PandasDataset(
        structure="DataStructure=MD:LEI_DATA(1.0)", data=data
    )
    return write([dataset])


def __clean_output(output, header=False):
    """Currently may add some extra lines in windows, 
    just removing the  CR character.
    We also clean the extra headers for chunking."""
    out_lst = output.splitlines()
    if not header:
        out_lst = out_lst[1:]
    output = "\n".join(out_lst)
    del out_lst
    return output


def streaming_load_save_csv_file(golden_copy_original_path, output_filename,
                                 use_sdmx_csv=False, nrows=None):
    """Load data and rename using small memory"""
    chunksize = None
    if nrows is None or nrows > 100000:
        chunksize = 100000
    data = pd.read_csv(golden_copy_original_path, dtype=str,
                       chunksize=chunksize, nrows=nrows)
    # Add header only to the first chunk
    add_header = True
    # Removing the file if already present
    if os.path.exists(output_filename):
        os.remove(output_filename)
    number_of_lines_written = 0
    if isinstance(data, pd.DataFrame):
        data = [data]
    for chunk in data:
        chunk = _process_chunk(chunk)
        if add_header:
            header = True
            add_header = False
        else:
            header = False
        number_of_lines_written += len(chunk)
        if not use_sdmx_csv:
            chunk.to_csv(output_filename, mode="a", index=False, header=header)
        else:
            out = _save_as_sdmx_csv(chunk)
            out = __clean_output(out, header)
            with open(output_filename, "w", encoding="utf-8") as f:
                f.write(out)
    print(f"Number of lines written: {number_of_lines_written}")


streaming_load_save_csv_file(
    golden_copy_original_path="data_files/golden-copy-original.csv",
    output_filename="data_files/golden_copy_changed_10000_sdmx.csv",
    use_sdmx_csv=True, nrows=10000)

Number of lines written: 9553


# Reading the ConceptScheme on SDMX-ML 2.1 using read_sdmx

For this example,
we generated a DataStructure on FMR called LEI_DATA,
with Short URN: DataStructure=MD:LEI_DATA(1.0),
with the required codelists to be used for structural validation on FMR.

This structures is also available at SDMX_Structures/structures.xml file in this project,
or at the MeaningfulData FMR (fmr.meaningfuldata.eu).
Currently the library does not support SDMX-ML 3.0,
so we will read only the ConceptScheme and descendants (available at SDMX_Structures/concepts.xml).

To ensure we are able to validate the data correctly,
we extended the CL_AREA codelist from SDMX
to add a code that was present in the LEI Golden Copy.

The code below reads the ConceptScheme and descendants,
ensure you have installed the xml extra from pysdmx.

```bash
pip install pysdmx[xml]
```

In [37]:
from pysdmx.io import read_sdmx

structures_msg = read_sdmx("SDMX_Structures/concepts.xml")
# We can access the first concept scheme, or look for the short_urn
concept_scheme1 = structures_msg.get_concept_schemes()[0]
concept_scheme2 = structures_msg.get_concept_scheme(
    "ConceptScheme=MD:LEI_CONCEPTS(1.0)")

# Retrieving the Schema from FMR

We may use as well the FMR Webservices to download the Schema from FMR, using the FusionJSON format.

In [38]:
from pprint import pprint
from pysdmx.api.fmr import RegistryClient
from pysdmx.io.format import StructureFormat

client = RegistryClient(
    "https://fmr.meaningfuldata.eu/sdmx/v2", format=StructureFormat.FUSION_JSON
)
# Recommend to use debugger to see the response
schema = client.get_schema(
    "datastructure", agency="MD", id="LEI_DATA", version="1.0"
)
pprint(schema)

Schema(context='datastructure', agency='MD', id='LEI_DATA', components=[Component(id='LEI', required=True, role=<Role.DIMENSION: 'D'>, concept=Concept(id='LEI', uri=None, urn=None, name='LEI', description=None, dtype=<DataType.STRING: 'String'>, facets=None, codes=None, enum_ref=None, annotations=()), local_dtype=<DataType.STRING: 'String'>, local_facets=None, name='LEI', description=None, local_codes=None, attachment_level=None, array_def=None, urn=None), Component(id='POSTAL_CODE', required=False, role=<Role.MEASURE: 'M'>, concept=Concept(id='POSTAL_CODE', uri=None, urn=None, name='Postal Code', description=None, dtype=<DataType.STRING: 'String'>, facets=None, codes=None, enum_ref=None, annotations=()), local_dtype=None, local_facets=None, name='Postal Code', description=None, local_codes=None, attachment_level=None, array_def=None, urn=None), Component(id='COUNTRY_INCORPORATION', required=False, role=<Role.MEASURE: 'M'>, concept=Concept(id='COUNTRY_INCORPORATION', uri=None, urn=None

# Using VTL to validate the data with GLEIF data quality checks

The VTL language allows us to perform validations over the data,
with a business friendly syntax. 

For this purpose, at MeaningfulData we have developed a library called vtlengine,
which is able to run VTL scripts over data.

In this example,
we will use a VTL script
that performs validations based on the GLEIF data quality checks
(link) and a custom validation on Subcategory data.


Steps to use VTL from pysdmx:
1. Convert the Schema to a VTL DataStructure
2. Validate the data using VTL
3. Analyse the results

---------(Explanation on VTL validations usefulness,
validating more than one component. Quick overview on the code
using VTL Playground)---------

## Convert the Schema to a VTL DataStructure

This code converts the pysdmx.model Schema and DataStructureDefinition objects into a VTL datastructure,
using MeaningfulData internal format, usable only with vtlengine.
On pysdmx we will include this method
but it will generate the VTL 2.1 Standard datastructure.
Both options will be usable by the vtlengine library.

## Setting up the code

In [39]:
import json
from typing import Optional, Dict, Any, List
from pysdmx.model.dataflow import DataStructureDefinition, Component
from pysdmx.model import Role

VTL_DTYPES_MAPPING = {
    "String": "String",
    "Alpha": "String",
    "AlphaNumeric": "String",
    "Numeric": "String",
    "BigInteger": "Integer",
    "Integer": "Integer",
    "Long": "Integer",
    "Short": "Integer",
    "Decimal": "Number",
    "Float": "Number",
    "Double": "Number",
    "Boolean": "Boolean",
    "URI": "String",
    "Count": "Integer",
    "InclusiveValueRange": "Number",
    "ExclusiveValueRange": "Number",
    "Incremental": "Number",
    "ObservationalTimePeriod": "Time_Period",
    "StandardTimePeriod": "Time_Period",
    "BasicTimePeriod": "Date",
    "GregorianTimePeriod": "Date",
    "GregorianYear": "Date",
    "GregorianYearMonth": "Date",
    "GregorianMonth": "Date",
    "GregorianDay": "Date",
    "ReportingTimePeriod": "Time_Period",
    "ReportingYear": "Time_Period",
    "ReportingSemester": "Time_Period",
    "ReportingTrimester": "Time_Period",
    "ReportingQuarter": "Time_Period",
    "ReportingMonth": "Time_Period",
    "ReportingWeek": "Time_Period",
    "ReportingDay": "Time_Period",
    "DateTime": "Date",
    "TimeRange": "Time",
    "Month": "String",
    "MonthDay": "String",
    "Day": "String",
    "Time": "String",
    "Duration": "Duration",
}

VTL_ROLE_MAPPING = {
    Role.DIMENSION: "Identifier",
    Role.MEASURE: "Measure",
    Role.ATTRIBUTE: "Attribute",
}


def to_vtl_json(
        dsd: DataStructureDefinition, path: Optional[str] = None
) -> Optional[Dict[str, Any]]:
    """Formats the DataStructureDefinition as a VTL DataStructure."""
    dataset_name = dsd.id
    components = []
    NAME = "name"
    ROLE = "role"
    TYPE = "type"
    NULLABLE = "nullable"

    _components: List[Component] = []
    _components.extend(dsd.components.dimensions)
    _components.extend(dsd.components.measures)
    _components.extend(dsd.components.attributes)

    for c in _components:
        _type = VTL_DTYPES_MAPPING[c.dtype]
        _nullability = c.role != Role.DIMENSION
        _role = VTL_ROLE_MAPPING[c.role]

        component = {
            NAME: c.id,
            ROLE: _role,
            TYPE: _type,
            NULLABLE: _nullability,
        }

        components.append(component)

    result = {
        "datasets": [{"name": dataset_name, "DataStructure": components}]
    }
    if path is not None:
        with open(path, "w") as fp:
            json.dump(result, fp, indent=2)
        return None

    return result

## Perform the conversion

In [40]:
from pprint import pprint

vtl_datastructure = to_vtl_json(schema)
pprint(vtl_datastructure)

{'datasets': [{'DataStructure': [{'name': 'LEI',
                                  'nullable': False,
                                  'role': 'Identifier',
                                  'type': 'String'},
                                 {'name': 'POSTAL_CODE',
                                  'nullable': True,
                                  'role': 'Measure',
                                  'type': 'String'},
                                 {'name': 'COUNTRY_INCORPORATION',
                                  'nullable': True,
                                  'role': 'Measure',
                                  'type': 'String'},
                                 {'name': 'COUNTRY_HEADQUARTERS',
                                  'nullable': True,
                                  'role': 'Measure',
                                  'type': 'String'},
                                 {'name': 'CATEGORY',
                                  'nullable': True,
                   

## Validate the data using VTL (sample 10000)

## Setting up the code

In [41]:
import pandas as pd
from vtlengine import run


def _load_script(filename):
    with open(filename, "r") as f:
        script = f.read()
    return script

---------(Explanation on the code, overview on the VTL run method documentation)---------

## Running the VTL script

In [42]:
script = _load_script("vtl/validations.vtl")
data_df = pd.read_csv("data_files/golden_copy_changed_10000.csv")
datapoints = {"LEI_DATA": data_df}

validations_result = run(script=script, data_structures=vtl_datastructure,
                         datapoints=datapoints)
pprint(validations_result)

{'errors_count': Dataset(name='errors_count',
                         components={'errorlevel': {"name": "errorlevel", "data_type": "Number", "role": "Identifier", "nullable": false},
                                     'int_var': {"name": "int_var", "data_type": "Integer", "role": "Measure", "nullable": true}},
                         data=   errorlevel  int_var
0           1      NaN),
 'validation.postal_codes_errors': Dataset(name='validation.postal_codes_errors',
                                           components={'CATEGORY': {"name": "CATEGORY", "data_type": "String", "role": "Measure", "nullable": true},
                                                       'COUNTRY_HEADQUARTERS': {"name": "COUNTRY_HEADQUARTERS", "data_type": "String", "role": "Measure", "nullable": true},
                                                       'COUNTRY_INCORPORATION': {"name": "COUNTRY_INCORPORATION", "data_type": "String", "role": "Measure", "nullable": true},
                           

### Getting the total number of errors (sample 10000)

In [43]:
validations_result['errors_count'].data

,errorlevel,int_var
0,1,NaN


### Analysing data on Subcategory errors (sample 10000)

In [44]:
cols_to_analyse = ['CATEGORY', 'SUBCATEGORY', 'errorcode', 'errorlevel']
validations_result['validation.subcategories_errors'].data[cols_to_analyse]

,CATEGORY,SUBCATEGORY,errorcode,errorlevel
0,RESIDENT_GOVERNMENT_ENTITY,None,C1,1
1,RESIDENT_GOVERNMENT_ENTITY,None,C1,1
2,RESIDENT_GOVERNMENT_ENTITY,None,C1,1
3,RESIDENT_GOVERNMENT_ENTITY,None,C1,1


---------(Explanation on Subcategory errors)---------

## Using VTL to perform calculations

--- Explanation on VTL scripts for calculations, overview on the code ---

## Running the VTL script

In [45]:
script = _load_script("vtl/calculations.vtl")
data_df = pd.read_csv("data_files/golden_copy_changed_10000.csv")
datapoints = {"LEI_DATA": data_df}

calculations_result = run(script=script, data_structures=vtl_datastructure,
                         datapoints=datapoints)
pprint(calculations_result)

{'calculation.country_dimension': Dataset(name='calculation.country_dimension',
                                          components={'CATEGORY': {"name": "CATEGORY", "data_type": "String", "role": "Measure", "nullable": true},
                                                      'COUNTRY_HEADQUARTERS': {"name": "COUNTRY_HEADQUARTERS", "data_type": "String", "role": "Identifier", "nullable": false},
                                                      'COUNTRY_INCORPORATION': {"name": "COUNTRY_INCORPORATION", "data_type": "String", "role": "Identifier", "nullable": false},
                                                      'LEI': {"name": "LEI", "data_type": "String", "role": "Identifier", "nullable": false},
                                                      'str_var': {"name": "str_var", "data_type": "String", "role": "Measure", "nullable": false}},
                                          data=                       LEI COUNTRY_INCORPORATION COUNTRY_HEADQUARTERS  \
0     00

In [46]:
calculations_result['lei_statistics'].data

,COUNTRY,MEASURE,OBS_VALUE
0,US,NUMBER_INCORPORATED_ENTITIES,594
1,CZ,NUMBER_INCORPORATED_ENTITIES,53
2,CA,NUMBER_INCORPORATED_ENTITIES,16
3,KY,NUMBER_INCORPORATED_ENTITIES,88
4,IE,NUMBER_INCORPORATED_ENTITIES,36
...,...,...,...
148,NO,NUMBER_ENTITIES_DIFF_HQ,1
149,CA,NUMBER_ENTITIES_DIFF_HQ,1
150,BE,NUMBER_ENTITIES_DIFF_HQ,1
151,MH,NUMBER_ENTITIES_DIFF_HQ,1


---------(Explanation on the calculations)---------

# Generate SDMX file with the aggregated data

Generate a PandasDataset from vtlengine output
and use the SDMX-ML 2.1 Data write method from pysdmx.

## Setting up the code

In [48]:
from pysdmx.io.pd import PandasDataset

data = calculations_result['lei_statistics'].data
structure = "DataStructure=MD:LEI_AGGREGATE_STATISTICS(1.0)"
pd_dataset = PandasDataset(structure=structure, data=data)

### Write the SDMX-ML 2.1 file

In [54]:
from pysdmx.io.xml.sdmx21.writer.structure_specific import write

output = write([pd_dataset], prettyprint=False)

output

'<?xml version="1.0" encoding="UTF-8"?><mes:StructureSpecificData xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:mes="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message" xmlns:ss="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/structurespecific" xmlns:com="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common" xmlns:ns1="urn:sdmx:org.sdmx.infomodel.datastructure.DataStructure=MD:LEI_AGGREGATE_STATISTICS(1.0):ObsLevelDim:AllDimensions" xsi:schemaLocation="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message https://registry.sdmx.org/schemas/v2_1/SDMXMessage.xsd"><mes:Header><mes:ID>a0b2eec9-7f18-4258-b949-33d96a1ee214</mes:ID><mes:Test>true</mes:Test><mes:Prepared>2025-01-24T15:02:53</mes:Prepared><mes:Sender id="ZZZ"/><mes:Structure structureID="LEI_AGGREGATE_STATISTICS" namespace="urn:sdmx:org.sdmx.infomodel.datastructure.DataStructure==MD:LEI_AGGREGATE_STATISTICS(1.0)" dimensionAtObservation="AllDimensions"><com:Structure><Ref agencyID="MD" id="LEI

# Reading back the SDMX file using read_sdmx

In [52]:
from pysdmx.io import read_sdmx

data_msg = read_sdmx(output)
data_msg.data[0].data

,COUNTRY,MEASURE,OBS_VALUE
0,US,NUMBER_INCORPORATED_ENTITIES,594
1,CZ,NUMBER_INCORPORATED_ENTITIES,53
2,CA,NUMBER_INCORPORATED_ENTITIES,16
3,KY,NUMBER_INCORPORATED_ENTITIES,88
4,IE,NUMBER_INCORPORATED_ENTITIES,36
...,...,...,...
148,NO,NUMBER_ENTITIES_DIFF_HQ,1
149,CA,NUMBER_ENTITIES_DIFF_HQ,1
150,BE,NUMBER_ENTITIES_DIFF_HQ,1
151,MH,NUMBER_ENTITIES_DIFF_HQ,1
